In [1]:
import numpy as np
import pandas as pd
from Bio.PDB import *
from Bio.AlignIO import FastaIO
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib qt

In [70]:
## damage control ?? 2/22/24
SLIST = pd.read_csv('/Users/katie/nano/staple_list.csv', header=None)
SLIST = list(SLIST.loc[0])

# Calculate distances between staples

In [2]:
# I42
p = PDBParser()
structure = p.get_structure('input', "/Users/katie/nano/05_icosahedron_DX_42bp-multimodel.pdb")

/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 79995
  warnings.warn(
/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 81027
  warnings.warn(
/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 82116
  warnings.warn(
/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 83143
  warnings.warn(
/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 84232
  warnings.warn(
/Users/katie/opt/anaconda3/envs/nano3/lib/python3.10/site-packages/Bio/PDB/PDBParser.

In [27]:
# load in staple list
staples = pd.read_csv('/Users/katie/nano/05_icosahedron_DX_42bp.csv')
staples.columns = ['No', 'Type1', 'Type2', 'A12', 'Strand name', 'Length', 'Sequence']

staples.No += 1  # after this modification, NO corresponds to tiamat staple numbers!!
staples = staples.iloc[1:]  # remove scaffold entry
staples.index = list(staples.No)  # e.g. staples.LOC[2] accesses staple #2 on tiamat!

In [23]:
list(list(structure.get_chains())[0].get_residues())[:6]

[<Residue GUA het=  resseq=1 icode= >,
 <Residue ADE het=  resseq=2 icode= >,
 <Residue GUA het=  resseq=3 icode= >,
 <Residue CYT het=  resseq=4 icode= >,
 <Residue GUA het=  resseq=5 icode= >,
 <Residue CYT het=  resseq=6 icode= >]

In [25]:
staples.head(20)# No corresponds to tiamat staple numbers!!

,No,Type1,Type2,A12,Strand name,Length,Sequence
2,2,Staple,vertex,A1,05_icosahedron_M_42bp_1,57,TACTATACCTATGGCTTTTTGAAATTCTTAATCGGTATCCATTTT...
3,3,Staple,edge,A2,05_icosahedron_M_42bp_2,32,AAAATAGATATGTGTAAAAGGAACGTAGCGGT
4,4,Staple,vertex,A3,05_icosahedron_M_42bp_3,34,GGCGCTTTTTTAGGGCGCTGGCAAGTTGCTATGC
5,5,Staple,vertex,A4,05_icosahedron_M_42bp_4,51,TTTAATTTTTAGTTCTGCTATTGACGGGGAATTTTTAGCCGGCGA...
6,6,Staple,edge,A5,05_icosahedron_M_42bp_5,32,AGCGAAAGCGTGGCGAGAAAGGAACGTGAACC
7,7,Staple,vertex,A6,05_icosahedron_M_42bp_6,34,TATCAGTTTTTGGCGATGGCCCACTAGGGAAGAA
8,8,Staple,vertex,A7,05_icosahedron_M_42bp_7,51,AAATCTTTTTAAAAGAATAGGAACAAGAGTCTTTTTCACTATTAA...
9,9,Staple,edge,A8,05_icosahedron_M_42bp_8,47,TGGCGTCAAAGGGCGAAAAAGAACGTGGACTCCAACTAACGCAAGCA
10,10,Staple,edge,A9,05_icosahedron_M_42bp_9,47,ACTTGTGTTGTTCCAGTTTGACCGAGATAGGGTTGAGGTCAGTTATC
11,11,Staple,vertex,A10,05_icosahedron_M_42bp_10,57,GCATCGTAGCGAAGGTTTTTATTAGTGACGGACATCTAGTGTTTT...


In [28]:
plt.scatter(staples.No, staples.Length)

In [5]:
staples[staples.Type2==' edge  ']

,No,Type1,Type2,A12,Strand name,Length,Sequence
3,3,Staple,edge,A2,05_icosahedron_M_42bp_2,32,AAAATAGATATGTGTAAAAGGAACGTAGCGGT
6,6,Staple,edge,A5,05_icosahedron_M_42bp_5,32,AGCGAAAGCGTGGCGAGAAAGGAACGTGAACC
9,9,Staple,edge,A8,05_icosahedron_M_42bp_8,47,TGGCGTCAAAGGGCGAAAAAGAACGTGGACTCCAACTAACGCAAGCA
10,10,Staple,edge,A9,05_icosahedron_M_42bp_9,47,ACTTGTGTTGTTCCAGTTTGACCGAGATAGGGTTGAGGTCAGTTATC
13,13,Staple,edge,A12,05_icosahedron_M_42bp_12,42,TATCTTCACGAGGTAGCAAATACGTATCTTCCCACAAGTAAA
19,19,Staple,edge,B6,05_icosahedron_M_42bp_18,42,AGTAACCTCAGCGCCCTTATTCCCTTTTTTGCAGTACTATGC
21,21,Staple,edge,B8,05_icosahedron_M_42bp_20,42,AGTTAGAATTGCGTTGCGCTCCTGCAGCAGGTACATGTTAGA
23,23,Staple,edge,B10,05_icosahedron_M_42bp_22,42,GCGCTTAATGCCAGCGGTAAGATCCTTGAGAGCACACCCGCC
25,25,Staple,edge,B12,05_icosahedron_M_42bp_24,32,CGAGTGGGGATGCTGAAGATCAGTGTGCTGCC
28,28,Staple,edge,C3,05_icosahedron_M_42bp_27,47,CCAAAAAAGCATCTTACGGATACTCACCAGTCACAGTGATGAGCACT


In [29]:
# subset to edge staples
staple_list = staples[staples['Type2'] == ' edge  '].No.values.tolist()  # accurate to tiamat

# manually append staples that are marked as vertex but are actually on the edge
staple_list += [16, 17, 44, 45, 47, 56, 64]

# initialize matrix of distances between staples
distances = pd.DataFrame(columns=staple_list, index=staple_list)

# initialize dictionary of x, y, z coordinates of staples (last atom in each staple)
coordinates = {}

In [30]:
# get sequences of all staples (not just edge)
LIST_STAPLES = []
for i in range(64):
    LIST_STAPLES.append(staples.iloc[i].Sequence[1:])
    
# get coordinates for each staple in staple_list
for i in range(1, len(list(structure.get_models())) ):  # start range from 1 since first "staple" in pdb is entire scaffold
    # get sequence of current item in pdb structure
    seq = ''.join(list(str(j)[9] for j in list(list(list(structure.get_models())[i].get_chains())[0].get_residues())))
    
    # get x,y,z coord of first atom in staple
    first_atom_coord = list(list(structure.get_models())[i].get_atoms())[0].get_coord()  # index [0] for 5' end!
    
    # KEY: find corresponding index of staple in given staple list, from tiamat
    staple = LIST_STAPLES.index(seq) + 2  # add +2 since first tiamat staple is considered #2

    coordinates[staple] = first_atom_coord.tolist()

In [31]:
# plot 3D scatterplot of coordinates

fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")

for staple in list(staples.No):
    x, y, z = coordinates[staple]
    ax.scatter3D(x, y, z, color = "green")
    ax.text(x,y,z,' ' + str(staple))

In [37]:
def dist(coord1, coord2):
    ''' Calculates euclidean distance between two 3D coords
    ''' 
    assert len(coord1) == len(coord2), 'Coordinates are different lengths?'
    return np.sqrt( (coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2 + (coord1[2] - coord2[2])**2 )

# distance is symmetric operation, so just calculate each distance once but fill in DataFrame twice
for _i in range(len(staple_list)):
    for _k in range(_i):
        i = staple_list[_i]
        k = staple_list[_k]
        distances.loc[i, k] = dist(coordinates[i], coordinates[k])
        distances.loc[k, i] = dist(coordinates[i], coordinates[k])
        
distances = distances.apply(pd.to_numeric).fillna(0)  # convert object type to floats so heatmap works, and replace nans with 0s

distances_nuc = distances / 3.4   # average nucleotide is 3.4 angstroms?? so create matrix of distances in terms of nucleotides

In [32]:
print( len(list(structure.get_models())) )  # each "model" is a staple
print( len(list(list(structure.get_models())[1].get_chains())) )  # there is only one chain, the NP itself i guess
print( len(list(list(list(structure.get_models())[1].get_chains())[0].get_residues())) )  # gives the nucleotides of a staple

65
1
32


In [33]:
STRUCTURE_STAPLES = []
for i in range(1,65):
    STRUCTURE_STAPLES.append(
        ''.join(list(str(i)[9] for i in list(list(list(structure.get_models())[i].get_chains())[0].get_residues()))) )

In [34]:
set(LIST_STAPLES) == set(STRUCTURE_STAPLES)

True

In [39]:
# plot distances between 30 edge staples (ico42)
sns.heatmap(distances, cmap='crest')
plt.show()

### Sanity check
I'm actually pretty sure my distances are calculated correctly?? Because when I go on ChimeraX and manually find the distance between two staples (e.g. I tested the distance between 34 and 83, and between 8 and 84), I actually get pretty much the same number of Angstroms calculated on ChimeraX as what I have calculated in my code (~400A for 34-83, ~100A for the 8-83)

# Determine if mRNAs fit inside origami

In [14]:
# load in mRNA sequences
root = '/Users/katie/nano/'
files = ['ofluc_excIRES.txt', 'ofluc_intIRES.txt', 'owas_excIRES.txt', 'owas_intIRES.txt',
        'fluc_ORF.txt', 'egfp_ORF.txt']

# descriptions:
# ofluc = firefly luciferase, circular DNA
# owas = wasabi (bathe lab's proprietary improved GFP), circular DNA
# fluc = firefly luciferase, linear (which we prefer now, and ofluc is no longer manufactured)
# IRES = internal ribosome entry site (helps with translation)
# ORF = open reading frame, the actual coding portion (as opposed to 5'UTR, for example)

mrna_dict = {}

for file in files:  # assumes files end in .txt, so name of mRNA is everything up to last 4 characters
    with open(root + file, 'r') as f:
        text = f.readlines()[0]
        if text[-1:] == '\n':
            text = text[:-1]
        mrna_dict[file[:-4]] = text
        
# lengths of mRNA sequences:
    # ofluc_excIRES = 1650 bp   excluding part of the sequence that we want to be exposed = IRES = ribosome-binding site
    # ofluc_intIRES = 1980 bp
    # owas_excIRES = 792 bp
    # owas_intIRES = 1320 bp
    # fluc_ORF = 1653 bp
    # egfp_ORF = 720 bp

In [15]:
# initial design (up to summer 2023) used mrna = mrna_dict['ofluc_excIRES']
mrna = mrna_dict['egfp_ORF']
def find_path(distances, starting_staple=56, staple_list=staple_list):
    # assume mrna binds initially at staple 3; see where it can go from there
    distance = 0  # initialize distance stretched by mrna to 0
    staples_used = [starting_staple]  # initialize first bound staple

    while set(staples_used) != set(staple_list):  # while we haven't used up all the staples
        neighbors_sorted = list(i for i in np.argsort(list(distances.loc[staples_used[-1]])))  # indexes of neighbors
        
        for neighbor in neighbors_sorted:
            if staple_list[neighbor] not in staples_used:   # if we come across an unused staple, it's the nearest unused one,
                staples_used.append(staple_list[neighbor])  # so we should add that to the shortest path
                break
                
        distance += distances.loc[staples_used[-1]][staples_used[-2]]  # gets distance between prior staple and current staple

    return staples_used, distance

In [ ]:
# initial design (up to summer 2023) used mrna = mrna_dict['ofluc_excIRES']
mrna = mrna_dict['fluc_ORF']
def find_path(distances, starting_staples=[3,6], staple_list=staple_list):
    # assume mrna binds initially at staple 3; see where it can go from there
    distance = 0  # initialize distance stretched by mrna to 0
    staples_used = starting_staples  # initialize first bound staple to be staples 3 and 6
    path3 = [starting_staples[0]]
    path6 = [starting_staples[1]]

    while set(staples_used) != set(staple_list):  # while we haven't used up all the staples
        neighbors_sorted = list(i for i in np.argsort(list(distances.loc[path3[-1]])))  # indexes of neighbors
        
        for neighbor in neighbors_sorted:
            if staple_list[neighbor] not in staples_used:   # if we come across an unused staple, it's the nearest unused one,
                staples_used.append(staple_list[neighbor])  # so we should add that to the shortest path
                path3.append(staple_list[neighbor])
                break
                
        distance += distances.loc[path3[-1]][path3[-2]]  # gets distance between prior staple and current staple
        
        
        neighbors_sorted = list(i for i in np.argsort(list(distances.loc[path6[-1]])))  # indexes of neighbors
        
        for neighbor in neighbors_sorted:
            if staple_list[neighbor] not in staples_used:   # if we come across an unused staple, it's the nearest unused one,
                staples_used.append(staple_list[neighbor])  # so we should add that to the shortest path
                path6.append(staple_list[neighbor])
                break
                
        distance += distances.loc[path6[-1]][path6[-2]]  # gets distance between prior staple and current staple

    return path3, path6, distance

In [ ]:
for i in range(len(path)):
    for k in range(i):
        path1, path2, distance = find_path(distances_nuc, starting_staples=[staple_list[i], staple_list[k]])
        staples_used = path1 + path2[::-1]

        if distance < 900:
            consecutive_distances = []
            for i in range(len(staples_used)-1):
                consecutive_distances.append(distances.loc[staples_used[i]][staples_used[i+1]])
            
            if max(consecutive_distances) < 136:
                print(distance)  # path length is 897bp; i chose starting staple 56 because yielded shortest path length
                print(staples_used)

                print(max(consecutive_distances))
                print()
                print()

In [16]:
staples_used, distance = find_path(distances_nuc)

print(distance)  # path length is 897bp; i chose starting staple 56 because yielded shortest path length
print(staples_used)

# ONLY IF DOING EGFP WHICH IS TOO SHORT AND USES 15 STAPLES OUT OF 30 POSSIBLE:
staples_used = staples_used[:15]

896.954090473171
[56, 64, 47, 61, 60, 29, 57, 25, 23, 28, 32, 34, 38, 36, 45, 41, 44, 46, 48, 51, 52, 19, 21, 16, 13, 17, 10, 9, 3, 6]


In [17]:
staples_used

[56, 64, 47, 61, 60, 29, 57, 25, 23, 28, 32, 34, 38, 36, 45]

In [ ]:
path1, path2, distance = find_path(distances_nuc, starting_staples=[56,29])
staples_used = path1 + path2[::-1]

print(distance)  # path length is 897bp; i chose starting staple 56 because yielded shortest path length
print(staples_used)

#### orders
* no overhang
* 20 base overhang
* 10 or 15bp overhang (maybe at a smaller scale so less expensive)
* like 4 T's prepended to 5' (then before that the anchor sequence)

* take ofluc seq, divide in 30 sections evenly 

In [23]:
rna_pieces = []
# for i in range(30):
#     # rna_pieces.append(mrna_dict['ofluc_excIRES'][55*i:55*(i+1)])   # 1650/30 = 55
#     rna_pieces.append(mrna_dict['fluc_ORF'][55*i - 4:55*(i+1)- 4])   # 1653/30 = 55
    
# ONLY FOR EGFP WHICH IS TOO SHORT AND ONLY USES 15 STAPLES OUT OF 30 POSSIBLE:
for i in range(15):
    rna_pieces.append(mrna_dict['egfp_ORF'][48*i:48*(i+1)])   # 720/15 = 36 using 15 staples

In [93]:
def revcomp(seq):
    ''' get reverse complement of a DNA sequence string '''
    code = {'A' : 'T', 'T' : 'A', 'C' : 'G', 'G' : 'C', 'U' : 'A'}
    revcomp = ''
    for base in seq[::-1]:
        revcomp += code[base]
    return revcomp

def off_by_one(seq1,seq2, errors=1):
    ''' checks if seqs of same length are exactly 1 base different 
        returns false IF THEY ARE MORE THAN 1 OFF
        can change errors to any int to detect off by (strictly) more than that many bases!
    '''
    count_diffs = 0
    for a, b in zip(seq1, seq2):
        if a!=b:
            if count_diffs == errors: 
                return False
            count_diffs += 1
    return True

In [128]:
prepends = {}

for anchor_len in [6]:#12,20]:  # testing 6nt overhangs on 12/3/2023
    prepends[anchor_len] = {}
    for i in range(len(rna_pieces)):
        staple = staples_used[i]

        # take first 12 or 20 or 6
        start = rna_pieces[i][2:anchor_len+2] # +2 seems to break up seq similarity

        # get revcomp and prepend to appropriate staple
        prepends[anchor_len][staple] = revcomp(start)

In [129]:
# analyze overhangs for orthogonality
prepends6 = list(prepends[6].values())

print(len(prepends6) == len(set(prepends6)))  # true; all overhangs unique

True


In [133]:
for i in range(15):
    for j in range(i,15):
        print(off_by_one(revcomp(prepends6[i]), prepends6[j], errors=1))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [87]:
# MODIFIED_STAPLES are the correct tiamat staples, after my manual 5' modifications
MODIFIED_STAPLES = pd.read_csv('/Users/katie/nano/30340329_Ico42_nicks_out staple-Table 1.csv')
MODIFIED_STAPLES = list(i[1:] for i in MODIFIED_STAPLES.loc[:,'Sequence '])

anchor_staple_seqs = {}

for anchor_len in [12,20]:
    anchor_staple_seqs[anchor_len] = {}
    for staple in staples_used:
        anchor_staple_seqs[anchor_len][staple] = prepends[anchor_len][staple] + 'TTTT' + MODIFIED_STAPLES[staple - 2]  # -2 for python indexing
        
# put finished anchor-staple sequences into a csv
columns = ['Well position', 'Name', 'Sequence', 'Length']

for anchor_len in [6]:#12,20]:
    staples_order = pd.DataFrame(columns=columns)
    staples_order['Well position'] = list(pd.read_csv('/Users/katie/nano/30340329_Ico42_nicks_out staple-Table 1.csv')['Well position '])[:30]
    for i in range(30):
        staple = sorted(staple_list)[i]
        if staple not in staples_used:
            continue
        staples_order.loc[i,'Name'] = staple
        staples_order.loc[i,'Sequence'] = anchor_staple_seqs[anchor_len][staple]
        staples_order.loc[i,'Length'] = len(anchor_staple_seqs[anchor_len][staple])
    #staples_order.to_csv(f'/Users/katie/Downloads/staple_order_{anchor_len}bp_EGFP.csv')


NameError: name 'staples_used' is not defined

In [89]:
MODIFIED_STAPLES_DF = pd.read_csv('/Users/katie/nano/30340329_Ico42_nicks_out staple-Table 1.csv')

In [100]:
MODIFIED_STAPLES_DF.head(10)

,Well position,Name,Sequence,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,A1,2,TACTATACCTATGGCTTTTTGAAATTCTTAATCGGTATCCATTTT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A2,3,ATGCAAAATAGATATGTGTAAAAGGAACGTAGCGGT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,4,GGCGCTTTTTTAGGGCGCTGGCAAGTTGCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A4,5,TTTAATTTTTAGTTCTGCTATTGACGGGGAATTTTTAGCCGGCGA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A5,6,AGAAAGCGAAAGCGTGGCGAGAAAGGAACGTGAACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A6,7,TATCAGTTTTTGGCGATGGCCCACTAGGGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A7,8,AAATCTTTTTAAAAGAATAGGAACAAGAGTCTTTTTCACTATTAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A8,9,TGGCGTCAAAGGGCGAAAAAGAACGTGGACTCCAACTAACGCAAGCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A9,10,ACTTGTGTTGTTCCAGTTTGACCGAGATAGGGTTGAGGTCAGTTATC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A10,11,GCATCGTAGCGAAGGTTTTTATTAGTGACGGACATCTAGTGTTTT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# calculate distances between consecutive staples
consecutive_distances = []
for i in range(len(staples_used)-1):
    consecutive_distances.append(distances_nuc.loc[staples_used[i]][staples_used[i+1]])
    
consecutive_distances

[27.28450448863282,
 27.196404991026483,
 33.14853476529787,
 29.019722965544613,
 30.23519705046049,
 29.007944739781895,
 55.16728908486299,
 29.01748815458811,
 27.149397440394225,
 28.363973275560035,
 31.279280859461327,
 27.167410614944043,
 35.53540582500542,
 46.91312913892897]

In [29]:
# plot 3D scatterplot of shortest path

fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")

x, y, z = coordinates[staples_used[0]]
ax.scatter3D(x, y, z, color = "green")
ax.text(x,y,z,' ' + str(staples_used[0]))

for staple in staples_used[1:]:
    prev_x, prev_y, prev_z = x, y, z
    x, y, z = coordinates[staple]
    ax.scatter3D(x, y, z, color = "green")
    ax.text(x,y,z,' ' + str(staple))
    plt.plot([prev_x, x], [prev_y, y], [prev_z, z], 'bo', linestyle="--")

# I52 (older model, not using) and older meeting notes

In [ ]:
# ICO52
print(paths[0][1])   # mRNA length necessary for the path found with 1 nearest neighbor
print(paths[1][1])   # mRNA length necessary for the path found with 2 nearest neighbors (varies if you rerun)
print(paths[2][1])   # mRNA length necessary for the path found with 3 nearest neighbors (varies if you rerun)

# lengths of mRNA sequences:
    # ofluc_excIRES = 1650 bp   excluding part of the sequence that we want to be exposed = IRES = ribosome-binding site
    # ofluc_intIRES = 1980 bp
    # owas_excIRES = 792 bp
    # owas_intIRES = 1320 bp

### Notes from 03/20/2023
* Want to place overhangs facing inwards so that, when we attach anchor sites, they are faacing inwards and our mRNA will come in and bind inside

### Notes from 03/16/2023

- We don't know yet if we expect the scaffold to degrade and release RNA to be accessed, or if we expect everything to stay intact and for cellular machinery to come inside the origami and initiate translation there
- Could scale down to ico42 (side length 14nm, rather than 17nm)
- If length of wasabi is really too short, can look into commercial linear RNAs like mCherry
- For now, we'll stick with icosahedron architecture (but could potentially look into 6 helix bundles/other forms)
- Decide structures we want to try; order oligos with overhangs; gels, folding w and w/o RNA
- Do we load mRNA in from the start, or do we fold DNA first and then incubate with RNA?
  
  
- we will do lipofectamine

- First, need to determine which architectures and mRNAs look good to fold (next week ish so we can order staples)
- Staples will come within like 2 weeks from now, then I can start folding experiments

## Calculating paths for different architectures

* Anna listed the following architectures as the first I should try, since they're the easiest for us to fold and get staples for:
  * PB42
  * PB84
  * I42
  * I52
  * O42